# LLM Path Extractors for Knowledge Graph Construction

Source:

- [llamaindex](https://docs.llamaindex.ai/en/latest/examples/property_graph/Dynamic_KG_Extraction/?h=comparing)


In this notebook, we'll compare three different LLM Path Extractors from llama_index:

- SimpleLLMPathExtractor
- SchemaLLMPathExtractor
- DynamicLLMPathExtractor

We'll use a Wikipedia page as our test data and visualize the resulting knowledge graphs using Pyvis.


## 1. Setup


In [1]:
from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.indices.property_graph import (
    SimpleLLMPathExtractor,
    SchemaLLMPathExtractor,
    DynamicLLMPathExtractor,
)
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

import wikipedia

import os

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [25]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

# Set up global configurations
llm = OpenAI(temperature=0.0, model="gpt-4.1-nano")

Settings.llm = llm
Settings.chunk_size = 2048
Settings.chunk_overlap = 20

## 2. Fetch Some Raw Text from Wikipedia¶


In [4]:
def get_wikipedia_content(title):
    try:
        page = wikipedia.page(title)
        return page.content
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation page. Options: {e.options}")
    except wikipedia.exceptions.PageError:
        print(f"Page '{title}' does not exist.")
    return None

In [28]:
wiki_title = "Barack Obama"
content = get_wikipedia_content(wiki_title)

if content:
    document = Document(text=content, metadata={"title": wiki_title})
    print(
        f"Fetched content for '{wiki_title}' (length: {len(content)} characters)"
    )
else:
    print("Failed to fetch Wikipedia content.")

Fetched content for 'Barack Obama' (length: 85216 characters)


## 2. SimpleLLMPathExtractor


In [29]:
kg_extractor = SimpleLLMPathExtractor(
    llm=llm, max_paths_per_chunk=20, num_workers=4
)

simple_index = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes=False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

simple_index.property_graph_store.save_networkx_graph(
    name="./SimpleGraph.html"
)

Extracting paths from text: 100%|██████████| 11/11 [00:07<00:00,  1.55it/s]


In [30]:
simple_index.property_graph_store.get_triplets(
    entity_names=["Barack Obama", "Obama"]
)[:5]

[(EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '70afd867-ef2f-4719-b2f8-8366537dd566'}, name='Obama'),
  Relation(label='Began presidency with', source_id='Obama', target_id='68 percent approval rating', properties={'title': 'Barack Obama', 'triplet_source_id': '70afd867-ef2f-4719-b2f8-8366537dd566'}),
  EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '70afd867-ef2f-4719-b2f8-8366537dd566'}, name='68 percent approval rating')),
 (EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '70afd867-ef2f-4719-b2f8-8366537dd566'}, name='Obama'),
  Relation(label='Approval rating dropped to', source_id='Obama', target_id='Low 40s', properties={'title': 'Barack Obama', 'triplet_source_id': '70afd867-ef2f-4719-b2f8-8366537dd566'}),
  EntityNode(label='entity', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '70afd867-

## 3. DynamicLLMPathExtractor

### Without intial ontology :

Here, we let the LLM define the ontology on the fly, giving it full freedom to label the nodes as it best sees fit.


In [32]:
kg_extractor = DynamicLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    num_workers=4,
    # Let the LLM infer entities and their labels (types) on the fly
    allowed_entity_types=None,
    # Let the LLM infer relationships on the fly
    allowed_relation_types=None,
    # LLM will generate any entity properties, set `None` to skip property generation (will be faster without)
    allowed_relation_props=[],
    # LLM will generate any relation properties, set `None` to skip property generation (will be faster without)
    allowed_entity_props=[],
)

dynamic_index = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes=False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

dynamic_index.property_graph_store.save_networkx_graph(
    name="./DynamicGraph.html"
)

dynamic_index.property_graph_store.get_triplets(
    entity_names=["Barack Obama", "Obama"]
)

Parsing nodes: 100%|██████████| 1/1 [00:00<00:00, 55.48it/s]
Extracting and inferring knowledge graph from text:   9%|▉         | 1/11 [00:06<01:02,  6.27s/it]

Error during extraction: 1 validation error for EntityNode
name
  Input should be a valid string [type=string_type, input_value=['Vice President Joe Biden', 'Valerie Jarrett'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


Extracting and inferring knowledge graph from text: 100%|██████████| 11/11 [00:20<00:00,  1.91s/it]


[(EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '2470e600-aecc-4be0-9211-6ed3fcc23bf5'}, name='Obama'),
  Relation(label='HAILED', source_id='Obama', target_id='the agreement', properties={'title': 'Barack Obama', 'triplet_source_id': 'b265004e-07a6-4240-9895-6207ce68d306'}),
  EntityNode(label='EVENT', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'b265004e-07a6-4240-9895-6207ce68d306'}, name='the agreement')),
 (EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '2470e600-aecc-4be0-9211-6ed3fcc23bf5'}, name='Obama'),
  Relation(label='ESTABLISHED', source_id='Obama', target_id='Obama Presidential Center', properties={'title': 'Barack Obama', 'triplet_source_id': '2470e600-aecc-4be0-9211-6ed3fcc23bf5'}),
  EntityNode(label='PRESIDENTIAL_LIBRARY', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '2470e600-aecc-4be0-9211-6ed3fcc23b

### With initial ontology for guided KG extraction :

Here, we have partial knowledge of what we want to detect, we know the article is about Barack Obama, so we define some entities and relations that could help guide the LLM in the labeling process as it detects the entities and relations. This doesn't guarantee that the LLM will use them, it simply guides it and gives it some ideas. It will still be up to the LLM to decide whether it uses the entities and relations we provide or not.


In [ ]:
kg_extractor = DynamicLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    num_workers=4,
    allowed_entity_types=["POLITICIAN", "POLITICAL_PARTY"],
    allowed_relation_types=["PRESIDENT_OF", "MEMBER_OF"],
    allowed_relation_props=["description"],
    allowed_entity_props=["description"],
)

dynamic_index_2 = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes=False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

dynamic_index_2.property_graph_store.save_networkx_graph(
    name="./DynamicGraph2.html"
)
dynamic_index_2.property_graph_store.get_triplets(
    entity_names=["Barack Obama", "Obama"]
)[:5]


Parsing nodes: 100%|██████████| 1/1 [00:00<00:00, 47.63it/s]
Extracting and inferring knowledge graph from text:  64%|██████▎   | 7/11 [00:23<00:12,  3.01s/it]

Error during extraction: 1 validation error for EntityNode
name
  Input should be a valid string [type=string_type, input_value=['Joe Biden', 'Valerie Jarrett'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


Extracting and inferring knowledge graph from text: 100%|██████████| 11/11 [00:46<00:00,  4.22s/it]


[(EntityNode(label='POLITICIAN', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'c53fbaad-027f-4c86-87e8-8a68ef7d0a62'}, name='Obama'),
  Relation(label='HAILED', source_id='Obama', target_id='the agreement', properties={'title': 'Barack Obama', 'triplet_source_id': 'c53fbaad-027f-4c86-87e8-8a68ef7d0a62'}),
  EntityNode(label='POLITICAL_AGREEMENT', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'c53fbaad-027f-4c86-87e8-8a68ef7d0a62'}, name='the agreement')),
 (EntityNode(label='POLITICIAN', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '9b6db1b1-c989-4ef3-93d8-7fe907887cd5'}, name='Barack Obama'),
  Relation(label='FAMILY_MEMBER', source_id='Barack Obama', target_id='Maya Soetoro-Ng', properties={'title': 'Barack Obama', 'triplet_source_id': 'b193b503-8127-4e7f-a35f-92aaf4c2d41f'}),
  EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': 'b193b503-8127-4e7f-a

## 4. SchemaLLMPathExtractor¶


In [35]:
from typing import Literal

# best practice to use upper-case
entities = Literal["PERSON", "PLACE", "ORGANIZATION"]
relations = Literal["HAS", "PART_OF", "WORKED_ON", "WORKED_WITH", "WORKED_AT"]

# define which entities can have which relations
validation_schema = {
    "PERSON": ["HAS", "PART_OF", "WORKED_ON", "WORKED_WITH", "WORKED_AT"],
    "PLACE": ["HAS", "PART_OF", "WORKED_AT"],
    "ORGANIZATION": ["HAS", "PART_OF", "WORKED_WITH"],
}


In [39]:
kg_extractor = SchemaLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    strict=True,  # Set to False to showcase why it's not going to be the same as DynamicLLMPathExtractor
    possible_entities=entities,
    possible_relations=relations,
    kg_validation_schema=validation_schema,
    possible_relation_props=None,
    possible_entity_props=None,
    num_workers=4,
)

schema_index = PropertyGraphIndex.from_documents(
    [document],
    llm=llm,
    embed_kg_nodes=False,
    kg_extractors=[kg_extractor],
    show_progress=True,
)

schema_index.property_graph_store.save_networkx_graph(
    name="./SchemaGraph.html"
)
schema_index.property_graph_store.get_triplets(
    entity_names=["Barack Obama", "Obama"]
)[:5]

Extracting paths from text with schema: 100%|██████████| 11/11 [00:28<00:00,  2.60s/it]


[(EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '062e0aaa-4f46-425d-bcaf-936c75bad72d'}, name='Barack Obama'),
  Relation(label='WORKED_ON', source_id='Barack Obama', target_id='Patriot Act', properties={'title': 'Barack Obama', 'triplet_source_id': '5bfdfbda-477a-4bad-98fa-0657c187e9ee'}),
  EntityNode(label='ORGANIZATION', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '5bfdfbda-477a-4bad-98fa-0657c187e9ee'}, name='Patriot Act')),
 (EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '4a0ac04e-9b2b-4600-a0ec-d8da67465b7b'}, name='Barack Obama Sr.'),
  Relation(label='HAS', source_id='Barack Obama Sr.', target_id='Barack Obama', properties={'title': 'Barack Obama', 'triplet_source_id': '4a0ac04e-9b2b-4600-a0ec-d8da67465b7b'}),
  EntityNode(label='PERSON', embedding=None, properties={'title': 'Barack Obama', 'triplet_source_id': '062e0aaa-4f46-425d-bcaf-936

## 5. Observations


### Comparison and Analysis

Let's compare the results of the three extractors:

- `SimpleLLMPathExtractor`: This extractor creates a basic knowledge graph without any predefined schema. It may produce a larger number of diverse relationships but might lack consistency in entity and relation naming.
- `DynamicLLMPathExtractor`:
  - This new extractor combines the flexibility of SimpleLLMPathExtractor with some initial guidance from a schema. It can expand beyond the initial entity and relation types, potentially producing a rich and diverse graph while maintaining some level of consistency.
  - Not giving it any entities or relations to start with in the input gives the LLM complete freedom to infer the schema on the fly as it best sees fit. This is going to vary based on the LLM and the temperature used.
- `SchemaLLMPathExtractor`: With a predefined schema, this extractor produces a more structured graph. The entities and relations are limited to those specified in the schema, which can lead to a more consistent but potentially less comprehensive graph. Even if we set "strict" to false, the extracted KG Graph doesn't reflect the LLM's pursuit of trying to find new entities and types that fall outside of the input schema's scope.


### Key observations:

- The SimpleLLMPathExtractor graph might have the most diverse set of entities and relations.
- The SchemaLLMPathExtractor graph should be the most consistent but might miss a lot of relationships that don't fit the predefined schema, even if we don't impose a strict validation of the schema.
- The DynamicLLMPathExtractor graph should show a balance between diversity and consistency, potentially capturing important relationships that the schema-based approach might miss while still maintaining some structure.

### The choice between these extractors depends on the specific use case:

- Use SimpleLLMPathExtractor for exploratory analysis where you want to capture a wide range of potential relationships for RAG applications, without caring about the entity types.
- Use SchemaLLMPathExtractor when you have a well-defined domain and want to ensure consistency in the extracted knowledge.
- Use DynamicLLMPathExtractor when you want a balance between structure and flexibility, allowing the model to discover new entity and relation types while still providing some initial guidance. This one is especially useful if you want a KG with labeled (typed) entities but don't have an input Schema (or you've partially defined the schema as a starting base).


## 6. Querying


In [40]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(model="text-embedding-3-large")

In [41]:
from llama_index.core.indices.property_graph import (
    LLMSynonymRetriever,
    VectorContextRetriever,
)


llm_synonym = LLMSynonymRetriever(
    dynamic_index_2.property_graph_store,
    llm=llm,
    include_text=False,
)
vector_context = VectorContextRetriever(
    dynamic_index_2.property_graph_store,
    embed_model=embed_model,
    include_text=True,
)

In [42]:
retriever = dynamic_index_2.as_retriever(
    sub_retrievers=[
        llm_synonym,
        vector_context,
    ]
)

In [45]:
nodes = retriever.retrieve("Who is Barack Obama?")

for node in nodes:
    print(node.text)

Obama -> HAILED -> the agreement
Obama -> RECOGNIZED -> celebrations across the US
Dmitry Medvedev -> REACHED_AGREEMENT_WITH -> Obama
New START treaty -> SIGNED_BY -> Obama
Obama -> APPOINTED -> Timothy Geithner
Obama -> STARTED -> nuclear arms buildup program
Obama -> SERVED_ON_COMMITTEE -> Foreign Relations Committee
Obama -> CALLED_WAR_IN_IRAQ_A_DANGEROUS_DISTRACTION -> War in Iraq
Obama -> PLANNED_TO_KEEP -> U.S. Forces in Afghanistan indefinitely
Obama administration -> WENT_AHEAD_WITH -> Bush administration program
Obama -> REPLACED_McChrystal -> David Petraeus
Obama -> EXPANDED -> federal lands and waters
Obama -> AUTHORIZED -> support to Saudi Arabia in Yemen
Obama -> SIGNED -> Budget Control Act of 2011
Obama -> WON -> 2008 Presidential Election
Obama -> ENDORSED -> Joe Biden
Obama -> ELECTED_AS -> President of the United States
Obama -> MADE_FIRST_DAY_DECISION -> Order withdrawal from Iraq
Obama -> CRITICIZED -> Russia's law discriminating against homosexuals
Obama -> REPLACE

We can also create a query engine with similar syntax.


In [50]:
query_engine = dynamic_index_2.as_query_engine(
    sub_retrievers=[
        llm_synonym,
        vector_context,
    ],
    llm=OpenAI(model="gpt-4.1"),
)

response = query_engine.query("Who is Barack Obama?")

print(str(response))

Barack Obama is the President of the United States of America who won the 2008 Presidential Election and was elected as President. He signed the New START treaty, appointed Timothy Geithner, and endorsed Joe Biden. He made significant decisions such as ordering the withdrawal from Iraq on his first day, proposing health care reform legislation, and signing the Budget Control Act of 2011. Obama also created 25 new national monuments, expanded federal lands and waters, and sponsored the Federal Funding Accountability and Transparency Act of 2006. He served on the Foreign Relations Committee, delivered a speech at Cairo University, and won the Audio Publishers Association Award. Obama authorized support to Saudi Arabia in Yemen, ordered a drone strike in Yemen, and replaced military commanders including General David D. McKiernan and David Petraeus. He criticized Russia's law discriminating against homosexuals and President Trump, and vetoed the Keystone XL pipeline bill. Obama hailed an 

In [51]:
response = query_engine.query("Where did Barack Obama work?")

print(str(response))

Barack Obama worked as President of the United States of America and served on the Foreign Relations Committee.


In [52]:
response = query_engine.query("Where did Barack Obama study?")

print(str(response))

There is no information provided about where Barack Obama studied.
